# **IMPORTING NECESSARY LIBRARIES**

In [1]:
import numpy as np
import pandas as pd

# Finxing a random seed value for reproducibility
np.random.seed(0)

# **LINEAR REGRESSION MODEL**

In Linear Regression Model, We can write the relationship between independent and identically distributed (i.i.d) Response Varaibles (Y) and Covariate Varaibles (X) as:

$$ y_i = x_i^T \beta + \epsilon_i $$

where, $\beta_i,$ i = 1,...,p are the unknown parameters and $\epsilon_i$ is error of the equation & are i.i.d and independent of $x_i$ with $ E(\epsilon_i|x_i) = 0 $.

 The most commonly used estimate for $ \beta $ is the ordinary least-square (OLS) estimate that minimizes the sum of squared residuals

$$ OLS:  \hat{\beta} = \underset{\beta}{\mathrm{argmin}} \, \sum_{i=1}^{n} (y_i - x_i^T \beta)^2 $$

Least Median of Squares:
 The LMS estimates are found by minimizing the median of the squared residuals

$$LMS : \hat{\beta} = \underset{\beta}{\mathrm{argmin}} \, \mathrm{Med} (y_i - x_i^T \beta)^2$$

 Least Trimmed Squares:
 The LTS estimate is defined as
$$ LTS : \hat{\beta} = \underset{\beta}{\mathrm{argmin}} \, \sum_{i=1}^{q} r(i)(\beta)^2 $$

where $ r(1)(\beta)^2 \leq \ldots \leq r(q)(\beta)^2 $ are ordered squared residuals, $q = (\frac{n}{2} + 1)$ and $n$ is the number of samples



In [38]:
# Defining a Class for the Linear Regression Model
class LinearRegression:
    def __init__(self, method='OLS', l_r=0.01, iter=100):
        # Initializing the hyper-parameters of the Linear Regression Model like method of estimation, Learning_rate, Number of Iterations
        self.method = method
        self.l_r = l_r
        self.iter = iter
        # Initializing the parameters of the Linear Regression Model like beta, cost
        self.beta = None
        self.cost_hist = None
        self.beta_hist = None

    # Defining a method to compute the errors (costs) of the model parameters (beta) for each method of estimation
    def compute_cost(self, X, Y, beta):
        # Number of Samples
        N = len(Y)
        # Predictions of the Linear Regression Model given by the below dot product between Covariate Matrix and Parameters Matrix
        preds = X.dot(beta)
        # The difference between the actual output and the model’s predictions (Residuals)
        resids = np.square(Y - preds)
        # Defining Objective Cost (Loss) Functions for each Method of Estimation
        # For Ordinary Least Squares (OLS) Method of Estimation
        if self.method == 'OLS':
            cost = (1/(2*N)) * np.sum(resids)
        # For Least Median Squares (LMS) Method of Estimation
        elif self.method == 'LMS':
            cost = np.median(resids)
        # For Least Trimmed Squares (LTS) Method of Estimation
        elif self.method == 'LTS':
            resids_sorted = np.sort(resids, axis=0)
            q = int((N / 2) + 1)
            cost = np.sum(resids_sorted[:q])
        return cost

    # Defining a method to compute the gradients of the Cost (Loss) Function with respect to model parameters (beta) for each method of estimation
    def compute_gradient(self, X, Y, beta):
        # Number of Samples
        N = len(Y)
        # Prediction of the Linear Regression Model given by the below dot product between Covariate Matrix and Parameters Matrix
        pred = np.dot(X, beta)
        # Gradient of the Residuals term
        resid = pred - Y
        # Defining Gradients of the Cost (Loss) Function with respect to model parameters (beta) for each Method of Estimation
        # For Ordinary Least Squares (OLS) Method of Estimation
        if self.method == 'OLS':
            grads = (1/n) * X.T.dot(resid)
        # For Least Median Squares (LMS) Method of by using sub-gradients method
        elif self.method == 'LMS':
            med = np.median(resid)
            grads = X.T.dot(np.sign(resid - med))
        # For Least Trimmed Squares (LTS) Method of Estimation by using sub-gradients method
        elif self.method == 'LTS':
            resid_sorted = np.sort(resid, axis=0)
            q = int((n / 2) + 1)
            grads = X.T.dot(np.sign(resid - resid_sorted[q]))
        return grads

    # Defining a method to perform the gradientd descent algorithm of the Cost (Loss) Function with respect to model parameters (beta) for each method of estimation
    def gradient_descent(self, X, Y, beta):
        # Creating Arrays to store the beta and cost values at each iteration
        self.cost_hist = np.zeros(self.iter)
        self.beta_hist = np.zeros((self.iter, len(beta)))
        # Creating a for loop to loop over the number of iterations
        for i in range(self.iter):
            # Calculate the gradient of the Cost (Loss) function with respect to beta
            grads = self.compute_gradient(X, Y, beta)
            # Updating the value of the estimated parameter beta by subtracting the product of the learning rate and the gradients from the current value of beta.
            beta = beta - self.l_r*grads
            # storing the current values of beta and cost (loss) in the above defined arrays
            self.beta_hist[i,:] = beta.T
            self.cost_hist[i]  = self.compute_cost(X, Y, beta)
        return beta

    # Defining a method to estimate the unknown parameters (beta) of the model (beta) for each method of estimation
    def fit(self, X, Y):
        # Initializing the parameter vector beta with random values & with size of the number of features in X
        self.beta = np.random.randn(X.shape[1],1)
        # Optimizing the parameters beta by using the gradientd descent algorithm
        self.beta = self.gradient_descent(X, Y, self.beta)

# **TESTING ON SYNTHETIC DATA**

Considering the two covariate variables, $X_i$, where $i = 1,2$ following a standard normal distribution $N(0,1)$, and noise $ϵ$ from a normal distribution $N(0,1)$.
By Using OLS, LMS, and LTS methods to estimate the model's parameters for $N = 20$ and $N = 100$ observations with $R = 200$ realizations.
ALso, By Comparing the estimates using given metrics and present the results in tables for $N = 20$ and $N = 100$ separately.

Metrics for Comparison:

$${Mean Square Error (MSE)}:MSE_i = bias(\hat{\beta}_i)^2 + var(\hat{\beta}_i)$$

$${Robust Bias (RB)}: RB_i = median(\hat{\beta}_i) - \beta_i$$

$$ {Median Absolute Deviation (MAD)}: MAD_i = median(|\hat{\beta}_i -\beta_i|)$$

where $\hat{\beta}_i$ and $\beta_i$ are the parameter estimate and true parameter respectively, and $i = 0,1,...,p$ are the number of regressors.



In [39]:
# Defining a function to generate Synthetic Data of Covariate Matrix and Response Matrix
def generate_data(N, beta_true):
    #  Generating a 2D array pf Covariate Matirx X of size N x len(beta_true) following a standard normal distribution with mean = 0 and standard deviation = 1
    X = np.random.normal(0, 1, (N, len(beta_true)))
    #  Generating a 2D array of noise of size N following a standard normal distribution with mean = 0 and standard deviation = 1
    noise = np.random.normal(0, 1, (N, 1))
    #  Generating the Response (Dependent) variable Matrix Y in a linear regression model
    Y = np.dot(X, beta_true) + noise
    return X, Y

# Defining a function to calculate the 3 Metrics Defined above (MSE, RB, MAD) for all 3 methods of estimation (OLS, LMS, LTS)
def compute_metrics(beta_est, beta_true, R):
    # Calculating the Mean Square Error (MSE) by taking square of the difference between the true parameters from the estimated parameters, and the mean across the rows (axis = 1)
    MSE = np.mean((beta_est - beta_true.flatten())**2, axis=0)
    # Calculating the Robust Bias (RB) by getting the median of the estimated parameters across the rows (axis = 1) and subtracting the true parameters
    RB = np.median(beta_est, axis=0) - beta_true.flatten()
    # Calculating the Mean Absolute Deviation (MAD) by taking absolute value of the difference between the true parameters from the estimated parameters, and by computing median across the rows (axis = 1)
    MAD = np.median(np.abs(beta_est - beta_true.flatten()), axis=0)
    return MSE, RB, MAD

# Defining The Number of Samples
N_values = [20, 100]
# Defining The Number of Realisations
R = 200
# Defining The True (Actual) Values of the Parameters (Beta)
beta_true = np.array([[3], [5]])
# Defining The Lists to store the values of parameters and the metrics for each method of estimation
beta_results = []
metric_results = []

# Defining a For loop to loop over the different numbers of samples
for N in N_values:
    # Defining a Nested For Loop to loop over every 3 methods of estimation ('OLS', 'LMS', 'LTS')
    for method in ['OLS', 'LMS', 'LTS']:
        # intializing beta_estimator matrix for all realisations to shape of number of Realisations x Covariates
        beta_est = np.zeros((R, np.shape(X)[1]))
        # intializing cost (loss) value matrix for all realisations
        final_costs = np.zeros(R)
        # Defining a Nested For Loop to loop over every realisation
        for r in range(R):
            # Generating Synthetic Data for Covariate Matrix (X) and Response Matrix (Y)
            X, Y = generate_data(N, beta_true)
            # Creating an instance of the LinearRegression class with a specified method of estimation
            lr = LinearRegression(method=method)
            # Fitting the linear regression model to the data X and Y using the specified method of estimation
            lr.fit(X, y)
            # Storing the estimated parameters from the fitted model for that realisation 'r'
            beta_est[r, :] = lr.beta.ravel()
        # Appending the average of the estimated parameters over all realisations for each method of estimation to the list
        beta_results.append([N, method, np.mean(beta_est[:,0]), np.mean(beta_est[:,1])])
        # Calculating the Mean Square Error (MSE), Robust Bias (RB), and Median Absolute Deviation (MAD) for the estimated parameters
        MSE, RB, MAD = compute_metrics(beta_est, beta_true, R)
        # Appending the Metrics for each method of estimation to the list
        metric_results.append([N, method, MSE[0], MSE[1], RB[0], RB[1], MAD[0], MAD[1]])

# Create DataFrames for beta results and metric results for each method of estimation
beta_cost_df = pd.DataFrame(beta_results, columns=['N', 'Method', 'Beta0', 'Beta1'])
metric_df = pd.DataFrame(metric_results, columns=['N', 'Method', 'MSE Beta0', 'MSE Beta1', 'RB Beta0', 'RB Beta1', 'MAD Beta0', 'MAD Beta1'])

# Print beta table for each method of estimation
print("Beta Table: for each method of estimation")
print(beta_cost_df.to_string(index=False))

# Print metric table for each method of estimation
print("\nMetric Table:  for each method of estimation")
print(metric_df.to_string(index=False))

Beta Table:
  N Method    Beta0    Beta1
 20    OLS 1.824189 3.006594
 20    LMS 2.952499 4.963279
 20    LTS 3.014799 4.973416
100    OLS 1.906855 3.157683
100    LMS 2.983220 5.005813
100    LTS 2.987761 5.014668

Metric Table:
  N Method  MSE Beta0  MSE Beta1  RB Beta0  RB Beta1  MAD Beta0  MAD Beta1
 20    OLS   1.870999   4.511711 -1.148117 -1.940122   1.148117   1.940122
 20    LMS   0.105870   0.107388 -0.022718 -0.061302   0.194628   0.205218
 20    LTS   0.085020   0.083951  0.025258 -0.019936   0.211906   0.187339
100    OLS   1.436542   3.623945 -1.036028 -1.803773   1.036028   1.803773
100    LMS   0.015706   0.019580  0.002052 -0.002674   0.085098   0.097420
100    LTS   0.019164   0.018548  0.014142  0.017511   0.086297   0.101667


# **PREPROCESSING THE REAL DATASET - MEDICAL_INSURANCE.CSV**

Changing Categorical Variables into Numerical Variables.
In the excel sheet,

* For 'sex' variable - I have replaced 'male' with '0' and 'female' with '1'.  
* For 'smoker' Variable - I have replaced 'yes' with '1' and 'no' with '0'.
* For 'region' Variable - I have replaced 'northeast' with '1' , 'northwest' with '2' , 'southeast' with '3' and 'southwest' with '4'.

In [58]:
# Creating a pandas Dataframe for the given Dataset
df = pd.read_csv('medical_insurance.csv')
df

,age,sex,bmi,children,smoker,region,charges
0,19,1,27.900,0,1,4,16884.92400
1,18,0,33.770,1,0,3,1725.55230
2,28,0,33.000,3,0,3,4449.46200
3,33,0,22.705,0,0,2,21984.47061
4,32,0,28.880,0,0,2,3866.85520
...,...,...,...,...,...,...,...
2767,47,1,45.320,1,0,3,8569.86180
2768,21,1,34.600,0,0,4,2020.17700
2769,19,0,26.030,1,1,2,16450.89470
2770,23,0,18.715,0,0,2,21595.38229


In [59]:
# Select split_ratio
split_ratio = 0.8

# Determining the training dataset size
total_rows = df.shape[0]
train_size = int(total_rows*split_ratio)

# Split data into test and train
train = df[0:train_size]
test = df[train_size:]

In [61]:
# Creating Covariate Matrix (X_train) = (age,sex,bmi,children,smoker,region) for Training Dataset
X_train = train.iloc[:, :-1]
print(X_train)

# Converting Data Frame to a Numpy Array
X_train = np.array(train.iloc[:, :-1])

      age  sex     bmi  children  smoker  region
0      19    1  27.900         0       1       4
1      18    0  33.770         1       0       3
2      28    0  33.000         3       0       3
3      33    0  22.705         0       0       2
4      32    0  28.880         0       0       2
...   ...  ...     ...       ...     ...     ...
2212   56    0  31.790         2       1       3
2213   36    0  28.025         1       1       1
2214   41    0  30.780         3       1       1
2215   39    0  21.850         1       0       2
2216   63    0  33.100         0       0       4

[2217 rows x 6 columns]


In [67]:
# Creating Response Matrix (Y_train) = (charges) for Training Dataset
Y_train = train.iloc[:, -1]
print(Y_train)

# Reshaping 1-Dimensional array to 2-Dimensions after converting Data Frame to a Numpy Array
Y_train = np.array(train.iloc[:, -1]).reshape(-1, 1)

0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
2212    43813.86610
2213    20773.62775
2214    39597.40720
2215     6117.49450
2216    13393.75600
Name: charges, Length: 2217, dtype: float64


In [62]:
# Creating Covariate Matrix (X_test) - (age,sex,bmi,children,smoker,region) for Testing Dataset
X_test = test.iloc[:, :-1]
print(X_test)

# Converting Data Frame to a Numpy Array
X_test = np.array(test.iloc[:, :-1])

      age  sex     bmi  children  smoker  region
2217   36    1  25.840         0       0       2
2218   28    1  23.845         2       0       2
2219   58    0  34.390         0       0       2
2220   36    0  33.820         1       0       2
2221   42    0  35.970         2       0       3
...   ...  ...     ...       ...     ...     ...
2767   47    1  45.320         1       0       3
2768   21    1  34.600         0       0       4
2769   19    0  26.030         1       1       2
2770   23    0  18.715         0       0       2
2771   54    0  31.600         0       0       4

[555 rows x 6 columns]


In [68]:
# Creating Response Matrix (Y_test) = (charges) for Testing Dataset
Y_test = test.iloc[:, -1]
print(Y_test)

# Reshaping 1-Dimensional array to 2-Dimensions after converting Data Frame to a Numpy Array
Y_test = np.array(test.iloc[:, -1]).reshape(-1, 1)

2217     5266.36560
2218     4719.73655
2219    11743.93410
2220     5377.45780
2221     7160.33030
           ...     
2767     8569.86180
2768     2020.17700
2769    16450.89470
2770    21595.38229
2771     9850.43200
Name: charges, Length: 555, dtype: float64


# **FITTING A LINEAR REGRESSION MODEL TO THE ABOVE DATASET**

Assuming a Linear Regression Model for the above Dataset, We can write the relationship between Response Varaibles (Y) and Regressor Varaibles (X) can be given as:

***$$ Y = \theta_0 X_0 + \theta_1 X_1 + \theta_2 X_2 + \theta_3 X_3 + \theta_4 X_4 + \theta_5 X_5 + \epsilon $$***

where, $\theta_i,$ i = 1,...,6 are the unknown parameters and $\epsilon_i$ is error of the equation & are i.i.d and independent of $x_i$ with $ E(\epsilon_i|x_i) = 0 $.

In [69]:
# Creating a list to store the parameters estimated for each method of estimation ('OLS', 'LMS', 'LTS')
theta_results = []

# For Loop to loop through all 3 methods of estimation ('OLS', 'LMS', 'LTS')
for method in ['OLS', 'LMS', 'LTS']:

    # intializing theta_estimator matrix to shape of number of covarities
    theta_est = np.zeros(((np.shape(X_train)[1])))
    # Estimating the parameters theta for Training Data using Linear Regression Model for each method of estimation with Gradient Descent
    # Creating an instance of the LinearRegression class with a specified method of estimation
    lr = LinearRegression(method=method)
    # Fitting the linear regression model to the given Training datasets X_train and Y_train using the specified method of estimation
    lr.fit(X_train, Y_train)
    # Storing the estimated parameters (theta) from the fitted model for each method of estimation
    theta_est[:,] = (lr.beta.ravel())
    # Appending the estimated parameters for each method to the list
    theta_results.append([method, theta_est[0], theta_est[1], theta_est[2], theta_est[3], theta_est[4], theta_est[5]])

# Create DataFrame for the estimated parameters (theta) for each method of estimation
theta_df = pd.DataFrame(theta_results, columns=['Method', 'Theta0', 'Theta1', 'Theta2', 'Theta3', 'Theta4', 'Theta5'])

# Print Theta table - The Table of Estimated Parameters by Each Method of Estimation
print("Theta Table: Estimated Parameters by Each Method")
print(theta_df.to_string(index=False))

Theta Table: Estimated Parameters by Each Method
Method         Theta0         Theta1         Theta2         Theta3         Theta4         Theta5
   OLS -8.990922e+142 -1.051283e+141 -6.624675e+142 -2.330659e+141 -4.208529e+140 -5.354742e+141
   LMS   3.347443e+02   2.398926e+01   8.134527e+00   2.120524e+02   4.470154e+02  -1.347535e+02
   LTS   3.383965e+02   2.230232e+01   1.222913e+01   2.141484e+02   4.463925e+02  -1.314360e+02


# **EVALUATING THE PERFORMANCE OF EACH METHOD BY MEAN SQUARE ERROR(MSE) ON TEST DATASET**

In [57]:
# Creating a list to store the Mean Square Errors(MSE) for each method of estimation ('OLS', 'LMS', 'LTS')
MSE_results = []

# Estimated parameters 'theta' for all 3 methods of estimation
theta_results_dict = {item[0]: item[1:7] for item in theta_results}
theta_OLS = theta_results_dict.get('OLS')
theta_LMS = theta_results_dict.get('LMS')
theta_LTS = theta_results_dict.get('LTS')

# Predicting Response variables (Y_test_bar) for the Covariate Test Dataset (X_test)
Pred_OLS = np.dot(X_test, theta_OLS).reshape(-1,1)
Pred_LMS = np.dot(X_test, theta_LMS).reshape(-1,1)
Pred_LTS = np.dot(X_test, theta_LTS).reshape(-1,1)

# Calculating Mean Squared Error between Predicted Responce Variables (Y_test_bar) vs True Response Variables (Y_test_bar)
N = np.shape(Y_test)[0]
MSE_OLS = (1 / N) * np.sum(((Y_test - Pred_OLS)**2), axis = 0)
MSE_LMS = (1 / N) * np.sum(((Y_test - Pred_LMS)**2), axis = 0)
MSE_LTS = (1 / N) * np.sum(((Y_test - Pred_LTS)**2), axis = 0)

# Appending the calculated MSE values for each method of estimation to the list
MSE_results.append([MSE_OLS[0], MSE_LMS[0], MSE_LTS[0]])

# Create DataFrame for the calculated MSE values for each method of estimation
MSE_df = pd.DataFrame(MSE_results, columns=['MSE_OLS', 'MSE_LMS', 'MSE_LTS'])

# Print MSE table - The Table of calculated MSE values for each method of estimation
print("MSE Table: Calculated Mean Square Error (MSE) Values for Each Method of Estimation")
print(MSE_df.to_string(index=False)))

MSE Table:
      MSE_OLS      MSE_LMS      MSE_LTS
3.182379e+289 1.368280e+08 1.367819e+08


# **FREQUENCY(FIRING RATE) VS INPUT CURRENT GRAPH**